处理less文件样式，并同步类名到同级目录下的index.tsx文件内.

1. 先查询到包含.less文件的目录
2. 格式化.less文件，以及目录下的tsx文件

### UI 代码
* button：点击后打开选择目录
* text：展示选中的目录路径

In [1]:
import tkinter as tk
from tkinter import filedialog
import ipywidgets as widgets
from IPython.display import HTML


def select_directory():
    # 初始化 Tkinter 主窗口
    root = tk.Tk()
    root.withdraw()  # 隐藏主窗口

    # 打开文件选择对话框
    folder_selected = filedialog.askdirectory()

    # 关闭 Tkinter 主窗口
    root.destroy()

    return folder_selected

button_dir = widgets.Button(
    description='选择查询目录',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='选择目录',
)

text = widgets.Text(
    value='',
    placeholder='查询目录',
    # description='选中的目录地址:',
    disabled=False
)

input = widgets.Text(
    value='',
    placeholder=' 要匹配的文件名称'
)
def btn_disable():
    button_dir.disabled = not button_dir.disabled

def dir_btn_click(b):
    btn_disable()
    text.value = select_directory()
    btn_disable()

button_dir.on_click(dir_btn_click)


### 解析配置
根据文件类型，使用特定的解析方式，按正则表达式规则做替换。

In [2]:
import re


# js 替换
# 将类名转换为小驼峰形式
def to_upperCase(match):
    return match.group(1).capitalize()


def camelCaseClassName(str):
    return re.sub(r"[-_]([a-z1-9]*)", to_upperCase, str)


def replace_class_name(match):
    class_name = match.group(1)
    class_text = match.group(2)
    list = [x for x in class_text.split(" ") if x]
    if len(list) > 1:
        came_case_str = ""
        for i in range(len(list)):
            if came_case_str:
                came_case_str = (
                    came_case_str + " " + f"${{styles.{camelCaseClassName(list[i])}}}"
                )
            else:
                came_case_str = f"${{styles.{camelCaseClassName(list[i])}}}"
        return f"{class_name}={{`{came_case_str}`}}"
    else:
        return f"{class_name}={{styles.{camelCaseClassName(list[0])}}}"


def replace_import(match):
    return "import styles from './index.module.less'"


# 转换 less 样式文件
def replace_less(match):
    groups = match.groups()
    match_text = groups[1]
    if not re.search(r"^ant", match_text):
        match_text = camelCaseClassName(groups[1])
    return groups[0] + match_text


def replace_global_less(match):
    groups = match.groups()
    if re.search(r"^\.ant", groups[1]):
        return groups[0] + groups[1] + groups[2]
    else:
        return groups[1]


def replace_antd(match):
    return match.group(1) + "@teno/antd-v4" + match.group(3)


def replace_antd_icon(match):
    return match.group(1) + "@teno/icons" + match.group(3)

def replace_antd_global(match):
    return ':global(.@{ant-prefix}' + match.group(1)+')'

js_patterns_and_replacements = {
    r'(className)=[\"\']([^"\']*)[\"\']': replace_class_name,
    r'(id)=[\"\']([^"\']*)[\"\']': replace_class_name,
    # r"import styled from [\"\']\./\w+\._?less[\"\']": replace_import,
    # r"<style jsx>{styled}</style>": "",
    r"import [\"\']\./\w+\.less[\"\']": replace_import,
    r"(from [\"\'])(antd)([\"\'])": replace_antd,
    r"(from [\"\'])(@ant-design/icons)([\"\'])": replace_antd_icon
}

less_patterns_and_replacements = {
    r"([.#])([\w-]+)": replace_less,
    r"(\:global\()(.+)(\))": replace_global_less,
    r"var\(--teno-text-color\)": "@color-text",
    r"var\(--teno-text-color-secondary\)": "#999",
    r"var\(--teno-text-color-tertiary\)": "#666",
    r"var\(--teno-text-color-quaternary\)": "#333",
    r"\.ant([\w-]+)": replace_antd_global,
}


"""
定义输出方式:
n: 创建新文件
w: 覆盖

n 模型下，文件默认命名，例：index.tsx -> index.new.tsx

"""

transform_config = [
    {
        "suffix": [".jsx", ".tsx", ".js"],
        "patterns_and_replacements": js_patterns_and_replacements,
        "is_format": False,
        "output": {"mode": "w", "name": ""},
    },
    {
        "suffix": [".less","._less"],
        "patterns_and_replacements": less_patterns_and_replacements,
        "is_format": False,
        "output": {"mode": "n", "name": "[name].module.less"},  # 默认[name][suffix]
    },
]

In [10]:
# 测试用
def replace_antd(match):
    return ':global(.@{ant-prefix}' + match.group(1)+')'


content = re.sub(
    r"\.ant([\w-]+)",
    replace_antd,
    """
      .ant-checkbox-wrapper + .ant-checkbox-wrapper {
        margin-left: 16px;
      }
      .ant-btn.resumeItemChat {}
""",
)

print(content)


      :global(.@{ant-prefix}-checkbox-wrapper) + :global(.@{ant-prefix}-checkbox-wrapper) {
        margin-left: 16px;
      }
      :global(.@{ant-prefix}-btn).resumeItemChat {}



使用 prettier 对文件进行格式化

In [3]:
# prettier 格式化文件
import subprocess
import os

def format_with_prettier(file_path,config_path):
    try:
        # 调用 Prettier 格式化文件
        result = subprocess.run(
            ['prettier', '--write', file_path,'--config', config_path],
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        print(f"Formatting successful: {result.stdout}")
    except subprocess.CalledProcessError as e:
        print(f"Error during formatting: {e.stderr}")

### 解析逻辑

In [6]:
from pathlib import Path
import re


# 读取文件
def readFileSync(file_path: Path):
    return file_path.read_text()


def writeFileSync(file_path: Path, content: str):
    return file_path.write_text(content)


# 生成
def generate_file(file_path: Path, config, content, format_config):
    output = config["output"]
    mode = output["mode"]
    is_format = config.get("is_format", False)

    if mode == "n":
        file_name = output.get("name") or "[name].new[suffix]"
        placeholders = {"[name]": file_path.stem, "[suffix]": file_path.suffix}

        for placeholder, value in placeholders.items():
            file_name = file_name.replace(placeholder, value)

        file_path = file_path.parent / file_name
        writeFileSync(file_path, content)
    elif mode == "w":
        writeFileSync(file_path, content)

    print("创建文件：" + str(file_path) + "完成")

    if is_format:
        format_with_prettier(str(file_path), str(format_config))

    return


# 解析文件
def transform_file(file_path: Path, config, format_config):
    file_content = readFileSync(file_path)
    if not file_content:
        return
    transformed_content = file_content
    for pattern, replacement in config["patterns_and_replacements"].items():
        transformed_content = re.sub(pattern, replacement, transformed_content)
    print("解析文件：" + str(file_path))
    generate_file(file_path, config, transformed_content, format_config)


# 处理文件
def processFile(file_path: Path, format_config):
    for item in transform_config:
        if file_path.suffix in item["suffix"]:
            transform_file(file_path.resolve(), item, format_config)
    return


# 查询less 同名js 文件，如果不存在则查询index文件
def search_js_file(file_path, suffixs):
    for suffix in suffixs:
        file_name = file_path.stem + suffix
        file_path = file_path.parent / file_name
        file_index_path = file_path.parent / ("index" + suffix)
        if file_path.is_file():
            return file_path
        elif file_index_path.is_file():
            return file_index_path
    return None


def search_format_file(dir_path, match_str, format_config):

    if dir_path.is_file():
        return

    if dir_path.is_dir():
        for file_path in dir_path.rglob("*"):
            if "node_modules" in file_path.parts:
                continue
            if file_path.is_file():
                if match_str in file_path.name:
                  # 覆写_less;更新 tsx或jsx后缀的文件
                  # js_file_path = search_js_file(file_path, [".tsx", ".jsx"])
                  # print("开始解析less文件")
                  # processFile(file_path, format_config)

                  print("开始解析js文件:",file_path)
                  processFile(file_path, format_config)
                       

    return

In [7]:

widgets.GridBox([button_dir,text,input],layout=widgets.Layout(grid_template_columns="160px 320px 240px"))

GridBox(children=(Button(button_style='success', description='选择查询目录', style=ButtonStyle(), tooltip='选择目录'), T…

In [8]:
path = Path(text.value)
search_format_file(path, input.value, path.parent / "./.prettierrc.js")


开始解析js文件: /Users/liepin/liepin-project/fe-nlpt-pc/v7/src/views-compatible/ResstoreSoV2/components/SuggestList/index.tsx
解析文件：/Users/liepin/liepin-project/fe-nlpt-pc/v7/src/views-compatible/ResstoreSoV2/components/SuggestList/index.tsx
创建文件：/Users/liepin/liepin-project/fe-nlpt-pc/v7/src/views-compatible/ResstoreSoV2/components/SuggestList/index.tsx完成
